# Tech Intro

## Main tables

In [789]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [790]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = deal_list[10] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [791]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 02:21:29.993331 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 02:21:29.994007 44470 sched.cpp:241] Version: 1.11.0
I1120 02:21:29.994235 10544 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 02:21:29.994309 10544 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 02:21:29.995682 10542 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75625
22/11/20 02:21:30 WARN HttpChannel: /jobs/
java.util.NoSuchElementException: Failed to get the application information. If you are starting up Spark, please wait a while until it's ready.
	at org.apache.spark.status.AppStatusStore.applicationInfo(AppStatusStore.scala:51)
	at org.apache.spark.ui.jobs.AllJobsPage.render(AllJobsPage.scala:276)
	at org.apache.spark.ui.WebUI.$anonfun$attachPage$1(WebUI.scala:90)
	at org.apache.spark.ui.JettyUtils$$anon$1.doGet(JettyUtils.scala:81)
	at javax.servlet.http.HttpServl

## Read the spec-file from Hadoop and show some data sample

In [792]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+------+-------------------+----+----+-------+---------+----------+--------------------------------------------------------------------------------------+---+
|id    |cad                |type|deal|deleted|filial_id|black_list|full_address                                                                          |lot|
+------+-------------------+----+----+-------+---------+----------+--------------------------------------------------------------------------------------+---+
|100002|73:24:040303:3462  |0   |0   |0      |28       |0         |Ульяновская, Ульяновск, Архитекторов 9 корп. 2, 4                                     |   |
|10001 |02:55:010601:394   |0   |0   |0      |1        |0         |Башкортостан, Уфа, Достоевского 99, 42                                                |   |
|10002 |02:55:010601:815   |0   |0   |0      |1        |0         |Башкортостан, Уфа, Достоевского 101, 6                                                |   |
|100024|72:24:0304013:5469 |0   |0   |0      |

## Show columns' properties

In [793]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- cad: string (nullable = true)
 |-- type: string (nullable = true)
 |-- deal: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- filial_id: string (nullable = true)
 |-- black_list: string (nullable = true)
 |-- full_address: string (nullable = true)
 |-- lot: string (nullable = true)



## Total rows (count)

In [794]:
eda_df.count()

177439

## Simple func to execute and show the result of the SQL-instruction

In [795]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [796]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [797]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|177439  |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [798]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,cad,type,deal,deleted,filial_id,black_list,full_address,lot
0,0,0,0,0,0,55,0,7974,0


In [799]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,cad,type,deal,deleted,filial_id,black_list,full_address,lot
0,0,1,177439,177439,176977,0,177335,0,0


In [800]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,cad,type,deal,deleted,filial_id,black_list,full_address,lot
0,0,11199,0,0,0,0,0,0,177439


In [801]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
cad,0,1,11199
type,0,177439,0
deal,0,177439,0
deleted,0,176977,0
filial_id,55,0,0
black_list,0,177335,0
full_address,7974,0,0
lot,0,0,177439


In [802]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [803]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
cad,0.000,0.001,6.311
type,0.000,100.000,0.000
deal,0.000,100.000,0.000
deleted,0.000,99.740,0.000
filial_id,0.031,0.000,0.000
black_list,0.000,99.941,0.000
full_address,4.494,0.000,0.000
lot,0.000,0.000,100.000


### SQL-instructions

In [804]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|177439            |
+------------------+

+-------------------+
|count(DISTINCT cad)|
+-------------------+
|148324             |
+-------------------+

+--------------------+
|count(DISTINCT type)|
+--------------------+
|1                   |
+--------------------+

+--------------------+
|count(DISTINCT deal)|
+--------------------+
|1                   |
+--------------------+

+-----------------------+
|count(DISTINCT deleted)|
+-----------------------+
|2                      |
+-----------------------+

+-------------------------+
|count(DISTINCT filial_id)|
+-------------------------+
|177                      |
+-------------------------+

+--------------------------+
|count(DISTINCT black_list)|
+--------------------------+
|2                         |
+--------------------------+

+----------------------------+
|count(DISTINCT full_address)|
+----------------------------+
|132332                      |
+--------

In [805]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [806]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute_template(col)

+-----+
|id   |
+-----+
|99999|
|99998|
|99997|
|99996|
|99995|
|99994|
|99993|
|99992|
|99991|
|99990|
|9999 |
|99989|
|99988|
|99987|
|99986|
|99985|
|99984|
|99983|
|99982|
|99981|
+-----+
only showing top 20 rows

None
+------+
|id    |
+------+
|100   |
|1000  |
|10000 |
|100000|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|10001 |
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
+------+
only showing top 20 rows

+----------------------------------------------------------------------------------+
|cad                                                                               |
+----------------------------------------------------------------------------------+
|№87 21кв.м. 8 этаж в комплексе Ручьи 2 очередь корп.2                             |
|№47:14:0312001:184                                                                |
|№ 78:36:0013262:693-78/039/2018-1  от 20.05.2018  (Собственность)                 |
|№ 56:47:0101015:408-56/0

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [807]:
spark.stop()

I1120 02:26:20.088167 10390 sched.cpp:2174] Asked to stop the driver
I1120 02:26:20.088843 10527 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75625
